In [72]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torchvision
torch.manual_seed(0) # Set for testing purposes, please do not change!

**NOTE:**
Unlike Basic GANs that you played with Nodes, in DCGANs you will play with Channels

# Information

1. We have NO any pooling layer
2. We have 2D batchnorm layer in both G and D.
3. We have NO fully connected hidden layer (nn.Linear).
4. ReLU in hidden layers - Tanh in final layer (Generators)
5. LeakyReLU in hidden layers - NO activation in final layer (Discriminator)
6. You will build a generator using 4 layers (3 hidden layers + 1 output layer)
7. You will use 3 layers in your discriminator's neural network

# Generator

In [86]:
def generator_block(C_in, C_out, K, S, final_layer=False):
    if final_layer:
        return nn.Sequential(
            nn.ConvTranspose2d(C_in, C_out, kernel_size = K, stride = S),
            nn.Tanh()
        )
    else:
        return nn.Sequential(
            nn.ConvTranspose2d(C_in, C_out, kernel_size = K, stride = S),
            nn.BatchNorm2d(C_out),
            nn.ReLU(inplace=True)
        )

In [87]:
class Generator(nn.Module):
    def __init__(self, C_noise, C_hidden, C_image):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            generator_block(C_noise   , 4*C_hidden, K=3, S=2),
            generator_block(4*C_hidden, 2*C_hidden, K=4, S=1),
            generator_block(2*C_hidden, C_hidden  , K=3, S=2),
            generator_block(C_hidden  , C_image   , K=4, S=2, final_layer=True)
        )

    def forward(self, x):
        return self.model(x)

# Discriminator

In [114]:
def discriminator_block(C_in, C_out, K, S, final_layer=False):
    if final_layer:
        return nn.Sequential(
            nn.Conv2d(C_in, C_out, kernel_size = K, stride = S),
        )
    else:
        return nn.Sequential(
            nn.Conv2d(C_in, C_out, kernel_size = K, stride = S),
            nn.BatchNorm2d(C_out),
            nn.LeakyReLU(inplace=True, negative_slope=0.2)
        )

In [126]:
class Discriminator(nn.Module):
    def __init__(self, C_image, C_hidden):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            discriminator_block(C_image   , C_hidden  , K=4, S=2),
            discriminator_block(C_hidden  , 2*C_hidden, K=4, S=2),
            discriminator_block(2*C_hidden, 1         , K=4, S=2, final_layer=True),
        )

    def forward(self, x):
        x=self.model(x)
        return x.view(x.shape[0], -1)

# Noise

In [127]:
def get_noise(N_noise, C_noise, device='cpu'):
    return torch.randn(N_noise, C_noise, device=device).view(-1, C_noise, 1, 1)

# Losses

In [128]:
def get_loss_dis(gen, dis,
                 real_image,
                 N_noise, C_noise,
                 criterion,
                 device):

    # some fake images will generate by Generator
    x_hat = gen(get_noise(N_noise, C_noise, device=device)).detach()

    # Discriminator determine how fake images are Fake (With Fake images)
    y_hat_fake = dis(x_hat)
    loss_fake = criterion(y_hat_fake, torch.zeros_like(y_hat_fake))

    # Discriminator determine how real images are Realistic (With Real images)
    y_hat_real = dis(real_image)
    loss_real = criterion(y_hat_real, torch.ones_like(y_hat_real))

    # Weighted Average
    loss_dis = (real_image.shape[0] * loss_real + N_noise * loss_fake) / (real_image.shape[0] + N_noise)

    return loss_dis

In [129]:
def get_loss_gen(gen, dis,
                 N_noise, C_noise,
                 criterion,
                 device):

    # some fake images will generate by Generator
    x_hat = gen(get_noise(N_noise, C_noise, device=device))

    # Discriminator determine how fake images are Fake (With Fake images)
    y_hat_fake = dis(x_hat)
    loss_gen = criterion(y_hat_fake, torch.ones_like(y_hat_fake))

    return loss_gen

# Helper Functions

In [130]:
def save_model(gen, dis, epoch, root):
    filename = root + f'\model_epoch_{epoch}.pt'
    torch.save({'epoch' : epoch,
                'model_dis_state_dict' : dis.state_dict(),
                'model_gen_state_dict' : gen.state_dict()},
               filename)

In [131]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

# Hyperparameters

In [132]:
# Sample numbers of noise and image
N_noise = 128
batch_size =128

# Channels of noise and image
C_noise = 64
C_image = 1

# Channels of hidden layers
C_hidden_gen = 64
C_hidden_dis = 16

# lr/epoch/disp
lr = 0.0002
epochs= 50
disp_freq=100

# roots
root_ds='D:\GitHub\gan-lab\Dataset'
root_models = "D:\GitHub\gan-lab\Models"

# devices
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Real Image

In [133]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

dataloader = DataLoader(
    MNIST(root_ds, download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True)

# Create Models / Criterion / Optimizers

In [134]:
# get instance from models
gen = Generator(C_noise, C_hidden_gen, C_image).to(device)
dis = Discriminator(C_image, C_hidden_dis).to(device)

# Loss function
criterion=nn.BCEWithLogitsLoss()

# Optimizers
optim_dis = torch.optim.Adam(dis.parameters(), lr=lr)
optim_gen = torch.optim.Adam(gen.parameters(), lr=lr)

# Test

In [135]:
for x, y in dataloader:
    x=x
    y=y
    break

In [136]:
image = get_noise(N_noise, C_noise, device=device)
gen_out = gen(image)
dis_out = dis(gen_out)
print(image.shape)
print(gen_out.shape)
print(dis_out.shape)

torch.Size([128, 64, 1, 1])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1])


# Train

In [137]:
loss_gen_min = np.Inf

for epoch in range(1,epochs+1):
    print(60 * "#")
    print(6 * "#" + " Epoch " + str(epoch) + " " + 45 * "#")
    print(60 * "#")

    # Set mode on "train mode"
    gen.train()
    dis.train()

    for real_image, _ in tqdm(dataloader):
        # GPU (model and data)
        real_image=real_image.to(device)

        # Discriminator Learning
        optim_dis.zero_grad()
        loss_dis = get_loss_dis(gen, dis, real_image, N_noise, C_noise, criterion, device)
        loss_dis.backward()
        optim_dis.step()

        # Generator Learning
        optim_gen.zero_grad()
        loss_gen = get_loss_gen(gen, dis, N_noise, C_noise, criterion, device)
        loss_gen.backward()
        optim_gen.step()

    #save_model(model_gen, model_dis, epoch, root_models)
    print("Loss Dis: {:.2f}\tLoss Gen: {:.2f}".format(loss_dis,loss_gen))

    gen.eval()
    fake_images = gen(get_noise(25, C_noise, device=device))
    show_tensor_images(fake_images)

############################################################
###### Epoch 1 #############################################
############################################################


  0%|          | 0/469 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 2.00 GiB total capacity; 50.20 MiB already allocated; 0 bytes free; 90.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF